In [1]:
import logging
import pandas as pd
import re
import requests
import pickle
import time
from pyquery import PyQuery
from urllib.parse import urljoin, urlparse
from models.model_issue import Issue

In [35]:
def get_url_page(url):
    response = requests.get(url)
    logging.info(response.status_code)

    if response.status_code == 200:
        return response.text
    else:
        if response.status_code == 429:
            print("Wait 3min "+url)
            time.sleep(180)
            response = requests.get(url)
            logging.info(response.status_code)
            if response.status_code == 200:
                return response.text
            elif response.status_code == 429:
                print("Wait 30min "+url)
                time.sleep(1800)
                response = requests.get(url)
                logging.info(response.status_code)
            else:
                #print(response.text) 
                raise Exception("get url page error: responce status code" + str(response.status_code))
        else:
            raise Exception("get url page error: responce status code" + str(response.status_code))



# 解析issue页，爬取页面的issue列表
def parse_repos_page(html, url, proj):
    #print("Parsing issue pages")

    # 用于存储所有issue信息
    issue_list_per_page = list()

    # 实例化PyQuery对象
    document = PyQuery(html)

    # 过滤获取issue所有标签
    issue_all = document('div').filter('.border-none')

    # items() 转化为list
    issue_items = issue_all.items()

    # 获取每一个issue
    for item in issue_items:
        head = item('a').filter('.h4')
        # issue信息项
        #id     = re.findall(r"\d+", item.attr('id'))[0]
        #title  = head.text()
        #type   = item('.IssueLabel').text()
        link   = urljoin(url, head.attr('href'))
        # time   = item('span').filter('.opened-by').children('relative-time').attr('datetime')
        author = item('.opened-by a').text()
        # 没有添加标签的类别，使用default代替
        #if type == '':
        #    type = 'default'

        #判断是issue还是pull,得到repo实际名称
        #i = 0
        #path = ""
        #while path!="issues" and path!="pulls":
        #    i+= 1
        #    path = urlparse(url).path.split('/')[i]
        #path = urlparse(url).path.split('/')[i-1]
        #issue_type = urlparse(url).path.split('/')[i]
        #path = urlparse(url).path.split('/')[1].lower()  #called source from now on
        #print(issue_type)
        # 使用json类型保存
        item_info = {
        #    'id'      :  proj + '#' + path + "#" + id,
        #    'source'  :  proj,
        #    'title'   :  title.replace("'", "''"),
        #    'type'    :  issue_type,
            'link'    :  link,
        #    'answered':  'no',
        #    'status'  :  status,
        #    'author'  :  author,
        #    'content' :  ''
        }
        #if issue_type!="issues":
        #    item_info['id'] = proj + '#' + path + "#" + issue_type + id
        
        #Low efficiency, moved to get_issue_comments
        #issue_status = get_status_page(link)
        #if issue_status != None :
        #    item_info['status'] = issue_status
        
        issue_list_per_page.append(item_info)
        #print(item_info['id'])

    logging.info(issue_list_per_page)

    # 下一页链接
    next_page_link = document('.next_page').attr('href')
    logging.info(next_page_link)

        # 判断下一页链接是否disabled
    if next_page_link != None:
        next_page_link = urljoin(url, next_page_link)

    return issue_list_per_page, next_page_link

In [36]:
url = "https://github.com/orgs/geotrellis/repositories?q=&type=source"
#df = pd.read_csv("/mnt/data0/proj_osgeo/gitrepository.csv")
    #print(df.head(5))
#proj = str(df.loc[df["Repos"] == url.replace('/pulls', '')]["Projects"].values[0]).lower()\
proj = url.split('/')[-2]

In [37]:

repos_list = []

html = get_url_page(url)
pull_list, next_page = parse_repos_page(html, url, proj)

while next_page != None:
    html = get_url_page(next_page)
    issue_list_per_page, next_page = parse_repos_page(html, url, proj)
    repos_list += issue_list_per_page

In [39]:
next_page

In [15]:
document = PyQuery(html)

In [20]:
issue_all = document('div').filter('.border-none')

In [22]:
issue_all.items()

<generator object PyQuery.items at 0x7fbd7d4f1f10>